In [1]:
import os
import pickle
import re
from string import punctuation
from stop_words import get_stop_words
from collections import defaultdict, Counter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
import nltk

from gensim.models import Word2Vec, Doc2Vec, FastText
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import warnings
warnings.filterwarnings('ignore')

/home/fassty/anaconda3/envs/deep_learning/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
display(os.listdir('saved_models/'))

exp_features = {
    'w2v_d2v_kwords_lemmatize': ['Klíčová slova'],
    'd2v_w2v_kwords_nolemma': ['Klíčová slova'],
    'ud_pipe_czech_text': ['Název česky', 'Anotace česky'],
    'ud_pipe_nazev_anotace_stopwords': ['Název česky', 'Anotace česky'],
    'tf_w2v_d2v_fast_eng_n_a_kwords_2': ['Název anglicky', 'Anotace anglicky', 'Klíčová slova'],
    'tf_w2v_d2v_fast_cz_nolemma_naz_anot_obor_uc_res_pos_2': ['Název česky', 'Anotace česky', 'Hlavní CEP obor', 'Podrobné informace o účastnících', 'Hlavní řešitelé', 'Poskytovatel']
}

['w2v_d2v_kwords_lemmatize',
 'tf_w2v_d2v_fast_cz_nolemma_naz_anot_obor_uc_res_pos_2',
 'ud_pipe_czech_text',
 'tf_w2v_d2v_fast_eng_n_a_kwords_2',
 'tf_w2v_d2v_fast_eng_n_a_kwords',
 'tf_w2v_d2v_fast_cz_nolemma_naz_anot_obor_uc_res_pos',
 'ud_pipe_nazev_anotace_stopwords',
 'd2v_w2v_kwords_nolemma']

In [3]:
query = widgets.Text(
    value='',
    placeholder='Enter a list of keywords separated by a comma',
    disabled=False,
    layout=widgets.Layout(height="auto", width="auto")
)
query

Text(value='', layout=Layout(height='auto', width='auto'), placeholder='Enter a list of keywords separated by …

In [106]:
pd.set_option('display.max_colwidth', None)

exp_name = 'ud_pipe_nazev_anotace_stopwords'
orig_df = pd.read_csv('../data/TACR_Starfos_isvav_project.csv')
df = orig_df[exp_features[exp_name]]
df = df[~df.isna().any(axis=1)]
orig_df = orig_df.iloc[df.index]

root_path = os.path.join('saved_models', exp_name)
with open(os.path.join(root_path, 'vectors.pickle'), 'rb') as handle:
    vectors = pickle.load(handle)

print('Loaded vectors')    
for model_name, model_vectors in vectors.items():
    print(model_name)
    if not model_name == 'fasttext_100_5':
        continue
    for vec in model_vectors:
        cos_sim = cosine_similarity(vec.reshape(1, -1), model_vectors)[0]
        idxs = cos_sim.argsort()[:-11:-1]
        temp_df = orig_df.iloc[idxs][['Název česky']].copy()
        temp_df['sim'] = cos_sim[idxs]
        temp_df['sim'] = temp_df['sim'].round(5)
        if temp_df['sim'].mean() >= 0.95:
            display(temp_df)

Loaded vectors
tf-idf_(1,1)
tf-idf_(1,2)
tf-idf_(1,3)
tf-idf_(1,4)
tf-idf_(1,5)
tf-idf_(2,2)
tf-idf_(2,3)
tf-idf_(2,4)
tf-idf_(2,5)
tf-idf_(3,3)
tf-idf_(3,4)
tf-idf_(3,5)
tf-idf_(4,4)
tf-idf_(4,5)
tf-idf_(5,5)
word2vec_25_5
word2vec_25_10
word2vec_25_25
word2vec_25_50
word2vec_50_5
word2vec_50_10
word2vec_50_25
word2vec_50_50
word2vec_100_5
word2vec_100_10
word2vec_100_25
word2vec_100_50
word2vec_200_5
word2vec_200_10
word2vec_200_25
word2vec_200_50
fasttext_25_5
fasttext_25_10
fasttext_25_25
fasttext_25_50
fasttext_50_5
fasttext_50_10
fasttext_50_25
fasttext_50_50
fasttext_100_5


,Název česky,sim
2,Molekulárně cytogenetická analýza buněk mozkových tumorů a její přínos pro diagnostiku a léčbu,1.00000
39874,Longitudinální a multidisciplinární péče o pacienty a rodiny s výskytem proximální svalové dystrofie,0.96040
40443,Spinocerebellární ataxie - komplexní diagnostika se zavedením molekulárních metod a následnou longitudiální multidisciplinární studií rodin pacientů.,0.95461
41773,Multidisciplinární přístup v diagnostice frontotemporálních lobárních degenerací a tauopatií: komplexní pohled na patogenetické mechanismy,0.95349
41438,Minimální uzlinové postižení u karcinomu žaludku.,0.95030
41275,Hereditární spinocerebelární ataxie - rozšíření molekulárně genetické diagnostiky s pokračující longitudinální multidisciplinární studií rodin pacientů,0.94930
42158,Úloha průtokové cytometrie a nových molekulárních metod v diagnostice dětských forem selhání kostní dřeně,0.94806
42755,Komplexní prognostický a prediktivní panel pro pacienty s chronickou lymfocytární leukémií: nástroj sekvenování nové generace vhodný pro klinickou praxi i studium genetického pozadí průběhu choroby,0.94691
41388,Vytvoření prognostického panelu u pacientů s monoklonální gamapatií nejasného významu s cílem zabránění transformace v maligní onemocnění.,0.94683
42486,Bioinformatické zpracováni NGS dat a funkční analýzy kandidátních variant pro testování hereditárních nádorových syndromů v ČR,0.94424


,Název česky,sim
8,Alterace genové exprese a signální transdukce v buňkách maligního melanomu. Vztah ke změnám stromálního mikroprostředí.,1.00000
958,Methylace histonu H3 v granulocytech jako prognostický marker remise chronické myeloidní leukémie,0.95548
41185,Kandidátní geny pro schizofrenii: vliv na fenotypovou variabilitu nemocných a in vitro manipulace genové exprese pomocí siRNA.,0.94959
39408,Genetické podklady humorální aktivace u chronického srdečního selhání,0.94955
41131,Charakteristika B lymfocytárních subpopulací u pacientů s běžným variabilním imunodeficitem,0.94881
40878,Prognostický význam přítomnosti hypokomplementemie pro typ a klinickou manifestaci lupusové glomerulonefritidy,0.94879
40204,Studium úlohy genetických faktorů v rozvoji neHodgkinských lymfomů,0.94861
39865,"Mitochondriální ""rozpřahující proteiny"" v játrech a kosterním svalu v průběhu perinatálního vývoje",0.94846
40726,Diagnostika neurodegenerativních svalových poruch pomocí analýzy mRNA/DNA vytypovaných genů a chromosomálních lokusů,0.94580
40888,"Prognostický a prediktivní význam imunohistochemické exprese hMLH1, hMSH2 a hMLH6 u pacientů s nemalobuněčným karcinomem plic",0.94521


,Název česky,sim
9,Vliv biotransformace a transportu na incidenci rakoviny prsu a účinky chemoterapie,1.00000
42269,Exosomální RNA produkovaná fibroblasty asociovanými s nádorem jako relevantní marker v prognóze nádorů hlavy a krku,0.96424
30908,Výzkum úlohy cholinergní neurotransmise při stárnutí a patogenezi Alzheimerovy nemoci,0.95705
50,Farmakogenomický přístup ke studiu účinnosti léčby u nemocných s roztroušenou sklerozou mozkomíšní,0.95260
24882,Mechanismy účinného zabíjení kmenových buněk rakoviny prsu,0.95162
42025,"Signální cesta Hedgehog-GLI: ukazatel biologického chování nádoru u melanomu, plicních a některých dalších nádorů, a její blokování jako protinádorová terapie.",0.94918
42615,Zkoumání získané chemorezistence pomocí profilovaní genů zahrnutých do opravy DNA u pacientů s rakovinou tlustého střeva,0.94764
16213,Alternativní mechanismy deregulace p53 dráhy u chronické lymfocytární leukémie,0.94729
41981,Studie mechanismu účinků biomarkerů korelujících s výsledkem léčby karcinomu prsu,0.94544
42756,Studium specifických podskupin u pacientů s adenokarcinomem pankreatu,0.94537


,Název česky,sim
14,"Multirezistentní bakteriální kmeny v chovech hospodářských zvířat - výskyt, možnosti šíření, analýza rizik pro zdravotní stav lidské populace a ochrana potravinového řetězce člověka",1.00000
46967,Fágová terapie infekcí vyvolaných Staphylococcus aureus v chovech hospodářských zvířat,0.95789
46224,Rezistence na antimikrobiální látky u mikroorganizmů vstupujících do potravního řetězce surovinami živočišného původu,0.95360
74,Zdravotně závažné mikroorganismy z prostředí zvířat v systému monitorování bezpečnosti potravin a v šíření antimikrobiální rezistence pro člověka,0.94737
47489,Účinnost kombinovaných probiotických preparátů ve výživě mláďat přežvýkavců,0.94376
46619,Vývoj nových nástrojů pro surveillance trichinelózy prasat a volně žijících zvířat v České republice,0.94362
5336,Probiotické baktérie střevní mikroflóry jako základ zdraví a pohody zvířat,0.94066
46852,Program zdravotní kontroly ekonomicky významných infekcí v chovech prasat,0.94003
42130,"Bakteriální rezistence k fluorochinolonům, možnosti její prevence a ochrana zdraví populace",0.93959
47437,Probiotika s autochtonní laktacidogenní flórou pro drůbež a prasata,0.93904


...

,Název česky,sim
167,Principy zpracování informace v neuronech a jejich aplikace,1.00000
24327,Úloha šumu při zpracování informace v neuronech,0.98831
15956,FoNeCo: Analytické základy neurovýpočtů,0.95866
13698,Efektivita informačního přenosu a role energetických omezení v neuronálních systémech,0.94495
20827,Stochastická shluková aktivita neuronů a kódování informace,0.94344
31059,Kódování intensity podnětu a zpracování informace v malých neuronových sítích. Teoretický přístup.,0.94299
31707,Alternativní učící algoritmy pro dopředné neuronové sítě,0.93727
28470,Využití umělých neuronových sítí při víceúrovňovém modelování transportních procesů v heterogenních materiálech,0.93435
27183,Distribuované dynamické odhadování v difuzních sítích,0.93343
28456,Informačně-teoretická analýza kódování stimulu v senzorických neuronech,0.93225


,Název česky,sim
170,Modelování a simulace náročných technických problémů: efektivní numerické algoritmy a paralelní implementace s pomocí nových informačních technologií,1.00000
16829,"Nové algoritmy pro přesnou, efektivní a robustní analýzu rozsáhlých systémů",0.96010
182,"Výzkum, testování, implementace a aplikace paralelních postupů pro modelování úloh mechaniky kontinua. Vytvoření elektronického výukového nástroje.",0.95113
10780,Algoritmy pro analýzu a návrh řízení složitých systémů,0.94676
26679,Výzkum simulačního a experimentálního modelování dynamiky mobilních kráčejících robotů,0.94667
17130,Vývoj a implementace paralelních algoritmů s aplikacemi v počítačové chemii.,0.94233
24344,Studium vlastností residuální aritmetiky pro řešení soustav lineárních rovnic,0.94199
36344,Účinné přístupy neurocomputing pro analýzu a posouzení konstrukcí,0.94086
26657,Numerické řešení pohybových rovnic s kontaktními okrajovými podmínkami a aplikace v nelineárních problémech vysokorychlostního impaktu,0.93940
10345,Nové přístupy k řízení složitých nelineárních dynamických podsystémů,0.93921


,Název česky,sim
182,"Výzkum, testování, implementace a aplikace paralelních postupů pro modelování úloh mechaniky kontinua. Vytvoření elektronického výukového nástroje.",1.00000
10713,Teoretické základy pro integrované řízení a optimalizaci technologických procesů,0.95323
170,Modelování a simulace náročných technických problémů: efektivní numerické algoritmy a paralelní implementace s pomocí nových informačních technologií,0.95113
10780,Algoritmy pro analýzu a návrh řízení složitých systémů,0.94811
26679,Výzkum simulačního a experimentálního modelování dynamiky mobilních kráčejících robotů,0.94565
10449,Vývoj a aplikace kontaktních úloh a kontaktní tvarová optimalizace v inženýrských problémech,0.94531
27126,Škálovatelné algoritmy založené na metodách rozložení oblastí pro výpočet dynamických kontaktních úloh a rázu těles se třením,0.94512
16829,"Nové algoritmy pro přesnou, efektivní a robustní analýzu rozsáhlých systémů",0.94395
38785,Optimální umisťování obslužných středisek pomocí IP-solverů,0.93930
24344,Studium vlastností residuální aritmetiky pro řešení soustav lineárních rovnic,0.93557


,Název česky,sim
184,Modelování a kvantifikace spolehlivosti dynamických systémů,1.00000
13742,Modelování dynamických finančních procesů se strukturálními změnami,0.96128
11368,Stochastické modelování interakce konstrukce s podložím (SISMO),0.95764
12454,Numerické modelování a analýza víceúrovňového porušení rozhraní v kompozitních materiálech,0.95746
24392,Inverzní problémy spolehlivosti konstrukcí,0.95645
21366,Vyhodnocování ekonomických rozhodovacích modelů a výsledků,0.95438
13865,Analýza konstrukcí tribun,0.95329
35432,Centrum Jaroslava Hájka pro teoretickou a aplikovanou statistiku,0.95212
21541,Stabilita a růst v dynamických modelech ekonomických systémů za neurčitosti,0.95110
16924,Modely náhodných bodových procesů a metody jejich statistické analýzy v řízení spolehlivosti systémů,0.95099


KeyboardInterrupt: 

In [109]:
exp_name = 'tf_w2v_d2v_fast_eng_n_a_kwords_2'
root_path = os.path.join('saved_models', exp_name)
with open(os.path.join(root_path, 'vectors.pickle'), 'rb') as handle:
    vectors = pickle.load(handle)
    
orig_df = pd.read_csv('../data/TACR_Starfos_isvav_project.csv')
df = orig_df[exp_features[exp_name]]
df = df[~df.isna().any(axis=1)]
orig_df = orig_df.iloc[df.index]
orig_df = orig_df.reset_index(drop=True)

vecs = vectors['word2vec_25_10']
idx = orig_df[orig_df['Název česky'] == '13 let jako významný milník v životě dětí - epidemiologická studie'].index
source = vecs[idx]
cos_sim = cosine_similarity(source, vecs)[0]
idxs = cos_sim.argsort()[:-11:-1]

temp_df = orig_df.iloc[idxs][['Klíčová slova', 'Název česky']].copy()
temp_df['Míra podobnosti'] = cos_sim[idxs]
temp_df['Míra podobnosti'] = temp_df['Míra podobnosti'].round(4)
temp_df.reset_index(drop=True)

,Klíčová slova,Název česky,Míra podobnosti
0,injury rate; morbidity; environment; lifestyle; smoking; health determinants; longitudinality; psychosocial aspects,13 let jako významný milník v životě dětí - epidemiologická studie,1.0000
1,injury rate; injury family; life style; smoking; life environment; health determinants; longitudinality; psycho-social aspects,15 let jako významný předěl v životě dětí - epidemiologická studie,0.9761
2,questionaires; longitudinality; health determinants; morbidity; accident rate; social development; family; life style,Vývoj během dětství v brněnském souboru WHO - ELSPAC. Vlastní šetření v 11 letech věku.,0.9738
3,childhood obesity; 21st century family; children; parents; health; lifestyle; nutrition; physical activity; motivation; change of attitudes; behavior change; barriers,"Vytvoření komplexního systému ochrany zdraví dětí a rodiny v kontextu změn 21. století se zaměřením na dětskou obezitu, výživu a pohyb",0.9677
4,quality of life; children; adolescents; illness; health; assessment; intervention,Kvalita života u dětí a dospívajících,0.9648
5,school youth; health related fitness; health state; dietary state; motor tests; functional state; mo,Školní mládež v konci dvacátého století,0.9598
6,deprivace; predikce; prevence; resilience; riziko,Ochrana duševního zdraví osob vystavených v dětství prokazatelnému psychosociálnímu riziku.,0.9590
7,CIDI; gravidita; interrupce; nechtěnost; poruchy; psychika,Analýza duševního zdraví osob ve středním věku narozených z nechtěných těhotenství.,0.9581
8,the health handicaps of children; the age structure of the child; the social security benefits,"Analýza zdravotních postižení dětí a jejich sociálních důsledků, úroveň sociální ochrany zdravotně postižených dětí, jejich výsledky a efektivnost",0.9574
9,very low birthweight; extremely low birthweight; bronchopulmonar dysplasia; gastroesophageal reflux; hyperexcitability,Follow-up of children with perinatal burden,0.9558


In [96]:
vectors.keys()

dict_keys(['tf-idf_(1,1)', 'tf-idf_(1,2)', 'tf-idf_(1,3)', 'tf-idf_(1,4)', 'tf-idf_(1,5)', 'tf-idf_(2,2)', 'tf-idf_(2,3)', 'tf-idf_(2,4)', 'tf-idf_(2,5)', 'tf-idf_(3,3)', 'tf-idf_(3,4)', 'tf-idf_(3,5)', 'tf-idf_(4,4)', 'tf-idf_(4,5)', 'tf-idf_(5,5)', 'word2vec_25_5', 'word2vec_25_10', 'word2vec_25_25', 'word2vec_25_50', 'word2vec_50_5', 'word2vec_50_10', 'word2vec_50_25', 'word2vec_50_50', 'word2vec_100_5', 'word2vec_100_10', 'word2vec_100_25', 'word2vec_100_50', 'word2vec_200_5', 'word2vec_200_10', 'word2vec_200_25', 'word2vec_200_50', 'fasttext_25_5', 'fasttext_25_10', 'fasttext_25_25', 'fasttext_25_50', 'fasttext_50_5', 'fasttext_50_10', 'fasttext_50_25', 'fasttext_50_50', 'fasttext_100_5', 'fasttext_100_10', 'fasttext_100_25', 'fasttext_100_50', 'fasttext_200_5', 'fasttext_200_10', 'fasttext_200_25', 'fasttext_200_50', 'doc2vec_25_5', 'doc2vec_25_10', 'doc2vec_25_25', 'doc2vec_25_50', 'doc2vec_50_5', 'doc2vec_50_10', 'doc2vec_50_25', 'doc2vec_50_50', 'doc2vec_100_5', 'doc2vec_100_1